In [380]:
import os
import pandas as pd

In [381]:
locations = set()
dialogue_text_options = set()
dialogue_npc_utterance = set()
dialogues = []

In [382]:
def get_dialogues(data):
    
    current_location = 'Null'
    current_talk = ''
    current_dialogue = ''
    for row in data:
        flag = False
        
        attribs = row.split('\n')[0]
        attribs = attribs.split('|')
        if attribs[0] == 'MOVE':
            for attrib in attribs:
                if 'cur-loc-' in attrib:
                    current_location = attrib.split('cur-loc-')[1]
        
        if attribs[0] == 'TALK':
            current_talk = attribs[3].split('cur-action-')[1]
            
            for row in data:
                attribs = row.split('\n')[0]
                attribs = attribs.split('|')
                
                if attribs[0] != 'DIALOG' and attribs[0] != 'PDAUSE' and flag == True:
                    break
                
                if attribs[0] != 'DIALOG':
                    continue
                
                flag = True
                if len(attribs) < 6:
                    current_dialogue += ('NPC: ' + attribs[-1] + '\n')
#                     print(attribs)
                else:
                    current_dialogue += ('PLR: ' + attribs[-1] + '\n')
#                     print(attribs)
                
            dialogues.add('Location: ' + current_location +', ' + 'Talk: ' + current_talk +'\n\n' + current_dialogue)
            current_dialogue = ''

In [383]:
def process_single_data_file(data):
    i = 0
    current_location = 'Null'
    for row in data:
        i += 1
        attribs = row.split('\n')[0]
        attribs = attribs.split('|')
        if attribs[0] == 'MOVE':
            for attrib in attribs:
                if 'prev-loc-' in attrib:
                    locations.add(attrib.split('prev-loc-')[1])
                if 'cur-loc-' in attrib:
                    current_location = attrib.split('cur-loc-')[1]
                    locations.add(attrib.split('cur-loc-')[1])
        
        if attribs[0] == 'DIALOG':
            if len(attribs) < 6:
                dialogue_npc_utterance.add(current_location +': '+ attribs[-1])
#                 dialogue_npc_utterance.add(attribs[-1])                
            else:
                dialogue_text_options.add(current_location +': '+ attribs[-1])
#                 dialogue_text_options.add(attribs[-1])
        

In [384]:
def file_name_extension(fname):
    name_ext = fname.rsplit('.', 1)
    return name_ext[0], name_ext[1]

In [385]:
def process_files(in_dir):
    file_count = 1
    for root, dirs, files in os.walk(in_dir):
        for file in files:
            fname, ext = file_name_extension(file)
            if ext == 'log' and '.' not in fname:
#                 print(file_count, ': ', os.path.join(root, file))
                with open(os.path.join(root, file), 'r') as data_file:
#                     process_single_data_file(data_file)
                    get_dialogues(data_file)
                    
            
                file_count += 1
#                 raise KeyboardInterrupt


In [386]:
locations = set()
dialogue_text_options = set()
dialogue_npc_utterance = set()
dialogues = set()

in_dir = './data/goal_recognition/Interaction Logs/'
# in_dir = './data/goal_recognition/test/'
process_files(in_dir)

In [387]:
len(dialogues)

562

In [390]:
i = 0
for d in dialogues:
    print(d)
    i += 1
    print('--------------------------------------------------')
    if i == 2:
        break

Location: infirmary-kim, Talk: talk-teresa

NPC: Uhhh, I think I ate something bad.
PLR: Can I ask about your illness?
NPC: Okay, but please be quick.
PLR: Nevermind.
PLR: I should get going.
NPC: Bye
PLR: Bye.

--------------------------------------------------
Location: infirmary-kim, Talk: talk-kim

NPC: Hey.
PLR: How can I help?
NPC: You can gather clues by talking to other team members, exploring the camp, and using the laboratory's testing equipment.   Use the W key to complete a diagnosis worksheet, then come talk to me once you have finished.
PLR: What do you know so far?
NPC: The illness spread through the camp a couple days ago.   So far, all we know is that it's either a pathogen, a mutagen, or a carcinogen.
PLR: Nevermind.
PLR: I should get going.
NPC: Come back when you have a diagnosis!
PLR: Bye.

--------------------------------------------------


In [395]:
out_file = './data/goal_recognition/test/conversation.txt'
with open(out_file, 'w') as wfile:
    for d in dialogues:
        wfile.write(d)
        wfile.write('--------------------------------------------------\n\n')